# Partitioning the input files

ROMS requires partitioned (or tiled) input files so that the simulation can be parallelized over multiple nodes. `ROMS-Tools` can create these partitioned files for you.

## Writing some example files
First we make some (non-partitioned) example files, which we will then partition in the next section. 

### Example grid file

In [1]:
from roms_tools import Grid

In [2]:
grid = Grid(
    nx=300,
    ny=150,
    size_x=23000,
    size_y=12000,
    center_lon=-161.0,
    center_lat=14.4,
    rot=-3.0,
)

In [3]:
grid.ds

<xarray.Dataset> Size: 5MB
Dimensions:       (eta_rho: 152, xi_rho: 302, xi_u: 301, eta_v: 151,
                   eta_coarse: 77, xi_coarse: 152, s_rho: 100, s_w: 101)
Coordinates:
    lat_rho       (eta_rho, xi_rho) float64 367kB -52.22 -52.06 ... 46.29 46.15
    lon_rho       (eta_rho, xi_rho) float64 367kB 114.6 115.2 ... 316.8 317.3
    lat_u         (eta_rho, xi_u) float64 366kB -52.14 -51.97 ... 46.35 46.22
    lon_u         (eta_rho, xi_u) float64 366kB 114.9 115.5 ... 316.5 317.1
    lat_v         (eta_v, xi_rho) float64 365kB -51.99 -51.83 ... 46.05 45.91
    lon_v         (eta_v, xi_rho) float64 365kB 114.4 115.0 ... 316.7 317.2
    lat_coarse    (eta_coarse, xi_coarse) float64 94kB -52.54 -52.2 ... 46.32
    lon_coarse    (eta_coarse, xi_coarse) float64 94kB 114.4 115.6 ... 317.7
Dimensions without coordinates: eta_rho, xi_rho, xi_u, eta_v, eta_coarse,
                                xi_coarse, s_rho, s_w
Data variables: (12/15)
    angle         (eta_rho, xi_rho) float64 367kB 0.4253 0.4253 ... -0.3405
    f             (eta_rho, xi_rho) float64 367kB -0.000115 ... 0.0001049
    pm            (eta_rho, xi_rho) float64 367kB 1.771e-05 ... 1.771e-05
    pn            (eta_rho, xi_rho) float64 367kB 2.23e-05 2.23e-05 ... 2.23e-05
    spherical     |S1 1B b'T'
    mask_rho      (eta_rho, xi_rho) int32 184kB 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    ...            ...
    mask_coarse   (eta_coarse, xi_coarse) int32 47kB 1 1 1 1 1 1 ... 1 1 1 1 1 1
    h             (eta_rho, xi_rho) float64 367kB 3.578e+03 ... 3.999e+03
    sigma_r       (s_rho) float32 400B -0.995 -0.985 -0.975 ... -0.015 -0.005
    Cs_r          (s_rho) float32 400B -0.992 -0.9753 ... -8.89e-05 -9.874e-06
    sigma_w       (s_w) float32 404B -1.0 -0.99 -0.98 -0.97 ... -0.02 -0.01 0.0
    Cs_w          (s_w) float32 404B -1.0 -0.9837 -0.9667 ... -3.95e-05 0.0
Attributes:
    title:               ROMS grid created by ROMS-Tools
    roms_tools_version:  2.6.3.dev18+gdb32644
    size_x:              23000
    size_y:              12000
    center_lon:          -161.0
    center_lat:          14.4
    rot:                 -3.0
    topography_source:   ETOPO5
    hmin:                5.0
    theta_s:             5.0
    theta_b:             2.0
    hc:                  300.0

In [4]:
grid.save("my_roms_grid.nc")

INFO - Writing the following NetCDF files:
my_roms_grid.nc


[PosixPath('my_roms_grid.nc')]

### Example surface forcing file

In [5]:
from roms_tools import SurfaceForcing
from datetime import datetime

In [6]:
start_time = datetime(2011, 1, 1)
end_time = datetime(2011, 1, 2)

In [7]:
era5_path = "/global/cfs/projectdirs/m4746/Datasets/ERA5/GLOBAL/reanalysis-era5-single-levels_2011-01.nc"

In [8]:
surface_forcing = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": era5_path},
    use_dask=True,
)

INFO - Data will be interpolated onto fine grid.
INFO - Applying 2D horizontal fill to the source data before regridding.
INFO - Applying 2D horizontal fill to the source data before regridding.


In [9]:
surface_forcing.ds

<xarray.Dataset> Size: 32MB
Dimensions:   (time: 25, eta_rho: 152, xi_rho: 302)
Coordinates:
    number    int64 8B 0
    expver    (time) <U4 400B dask.array<chunksize=(1,), meta=np.ndarray>
    abs_time  (time) datetime64[ns] 200B 2011-01-01 ... 2011-01-02
  * time      (time) float64 200B 4.018e+03 4.018e+03 ... 4.019e+03 4.019e+03
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    uwnd      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    vwnd      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    swrad     (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    lwrad     (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    Tair      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    rain      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    qair      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
Attributes:
    title:                 ROMS surface forcing file created by ROMS-Tools
    roms_tools_version:    2.6.3.dev18+gdb32644
    start_time:            2011-01-01 00:00:00
    end_time:              2011-01-02 00:00:00
    source:                ERA5
    correct_radiation:     True
    use_coarse_grid:       False
    model_reference_date:  2000-01-01 00:00:00
    type:                  physics

In [10]:
filepath_forcing = "my_surface_forcing.nc"

In [11]:
surface_forcing.save(filepath_forcing)

INFO - Writing the following NetCDF files:
my_surface_forcing_201101.nc


[########################################] | 100% Completed | 295.86 s


[PosixPath('my_surface_forcing_201101.nc')]

## Partitioning existing files
We will now partition the two files written in the previous section. However, note that the `partition_netcdf` tool in `ROMS-Tools` operates independently of whether the files were originally created using `ROMS-Tools`.

In [12]:
from roms_tools import partition_netcdf

We need to tell the `partition_netcdf` function what domain decomposition to use via the following two parameters:

* `np_eta` : The number of partitions along the `eta` direction (corresponding to `ny` in the grid).
* `np_xi` : The number of partitions along the `xi` direction (corresponding to `nx` in the grid).

In [20]:
%time partition_netcdf("my_roms_grid.nc", np_eta=3, np_xi=6)

CPU times: user 347 ms, sys: 104 ms, total: 451 ms
Wall time: 4.33 s


[PosixPath('my_roms_grid.00.nc'),
 PosixPath('my_roms_grid.01.nc'),
 PosixPath('my_roms_grid.02.nc'),
 PosixPath('my_roms_grid.03.nc'),
 PosixPath('my_roms_grid.04.nc'),
 PosixPath('my_roms_grid.05.nc'),
 PosixPath('my_roms_grid.06.nc'),
 PosixPath('my_roms_grid.07.nc'),
 PosixPath('my_roms_grid.08.nc'),
 PosixPath('my_roms_grid.09.nc'),
 PosixPath('my_roms_grid.10.nc'),
 PosixPath('my_roms_grid.11.nc'),
 PosixPath('my_roms_grid.12.nc'),
 PosixPath('my_roms_grid.13.nc'),
 PosixPath('my_roms_grid.14.nc'),
 PosixPath('my_roms_grid.15.nc'),
 PosixPath('my_roms_grid.16.nc'),
 PosixPath('my_roms_grid.17.nc')]

In [14]:
%time partition_netcdf("my_surface_forcing_201101.nc", np_eta=3, np_xi=6)

CPU times: user 274 ms, sys: 100 ms, total: 374 ms
Wall time: 3.85 s


[PosixPath('my_surface_forcing_201101.00.nc'),
 PosixPath('my_surface_forcing_201101.01.nc'),
 PosixPath('my_surface_forcing_201101.02.nc'),
 PosixPath('my_surface_forcing_201101.03.nc'),
 PosixPath('my_surface_forcing_201101.04.nc'),
 PosixPath('my_surface_forcing_201101.05.nc'),
 PosixPath('my_surface_forcing_201101.06.nc'),
 PosixPath('my_surface_forcing_201101.07.nc'),
 PosixPath('my_surface_forcing_201101.08.nc'),
 PosixPath('my_surface_forcing_201101.09.nc'),
 PosixPath('my_surface_forcing_201101.10.nc'),
 PosixPath('my_surface_forcing_201101.11.nc'),
 PosixPath('my_surface_forcing_201101.12.nc'),
 PosixPath('my_surface_forcing_201101.13.nc'),
 PosixPath('my_surface_forcing_201101.14.nc'),
 PosixPath('my_surface_forcing_201101.15.nc'),
 PosixPath('my_surface_forcing_201101.16.nc'),
 PosixPath('my_surface_forcing_201101.17.nc')]

## Ignoring Coarse Dimensions During Partitioning

The grid file generated by `ROMS-Tools` always includes the dimensions `eta_coarse` and `xi_coarse`. These dimensions, along with their associated variables, are only relevant when **coarse-resolution forcing** is used. This typically applies when surface forcing fields are generated on a grid coarser than the ROMS model grid, see the [ROMS-Tools documentation on surface forcing](https://roms-tools.readthedocs.io/en/latest/surface_forcing.html#Fine-vs.-coarse-grid). The coarse forcing configuration is activated when the ROMS `bulk_frc.opt` file includes the setting:
```bash
interp_frc = 1
```
With this option enabled, ROMS interpolates surface forcing data from the coarse grid onto the fine model grid at runtime.

If your simulation does **not** use coarse forcing, then the `eta_coarse` and `xi_coarse` dimensions in the grid file can be safely ignored, including during partitioning. This allows for a broader range of valid values for `np_eta` and `np_xi`, since only the fine grid dimensions must be divisible by these parameters. In contrast, if the coarse dimensions `eta_coarse`, `xi_coarse` are included, **both** fine and coarse grid sizes must be divisible by `np_eta` and `np_xi`.

The `partition_netcdf()` function provides a parameter `include_coarse_dimensions` to control this behavior.

### Example
Our grid above was created with:

* `nx = 300` (corresponding to `xi_rho`)
* `ny = 150` (corresponding to `eta_rho`)

By default, ROMS-Tools also includes coarse grid dimensions:

* `nx / 2 = 150` (corresponding to `xi_coarse`)
* `ny / 2 = 75` (corresponding to `eta_coarse`)

Now let’s try partitioning with:

* `np_xi = 12`
* `np_eta = 6`

These values divide the fine dimensions:

* `np_xi = 12` divides `nx = 300`
* `np_eta = 6` divides `ny = 150`

However, they do not divide the coarse dimensions:

* 12 does not divide 150

* 6 does not divide 75

As a result, if `include_coarse_dims = True` (the default), partitioning will fail with an error like:

In [24]:
%time partition_netcdf("my_roms_grid.nc", np_eta=6, np_xi=12, include_coarse_dimensions=True)

ValueError: Dimension 'eta_coarse' of size 75 cannot be evenly divided into 6 partitions.

To avoid this, you can set `include_coarse_dims = False` to skip partitioning along the coarse dimensions:

In [25]:
%time partition_netcdf("my_roms_grid.nc", np_eta=6, np_xi=12, include_coarse_dims=False)

CPU times: user 1.37 s, sys: 416 ms, total: 1.78 s
Wall time: 26.8 s


[PosixPath('my_roms_grid.00.nc'),
 PosixPath('my_roms_grid.01.nc'),
 PosixPath('my_roms_grid.02.nc'),
 PosixPath('my_roms_grid.03.nc'),
 PosixPath('my_roms_grid.04.nc'),
 PosixPath('my_roms_grid.05.nc'),
 PosixPath('my_roms_grid.06.nc'),
 PosixPath('my_roms_grid.07.nc'),
 PosixPath('my_roms_grid.08.nc'),
 PosixPath('my_roms_grid.09.nc'),
 PosixPath('my_roms_grid.10.nc'),
 PosixPath('my_roms_grid.11.nc'),
 PosixPath('my_roms_grid.12.nc'),
 PosixPath('my_roms_grid.13.nc'),
 PosixPath('my_roms_grid.14.nc'),
 PosixPath('my_roms_grid.15.nc'),
 PosixPath('my_roms_grid.16.nc'),
 PosixPath('my_roms_grid.17.nc'),
 PosixPath('my_roms_grid.18.nc'),
 PosixPath('my_roms_grid.19.nc'),
 PosixPath('my_roms_grid.20.nc'),
 PosixPath('my_roms_grid.21.nc'),
 PosixPath('my_roms_grid.22.nc'),
 PosixPath('my_roms_grid.23.nc'),
 PosixPath('my_roms_grid.24.nc'),
 PosixPath('my_roms_grid.25.nc'),
 PosixPath('my_roms_grid.26.nc'),
 PosixPath('my_roms_grid.27.nc'),
 PosixPath('my_roms_grid.28.nc'),
 PosixPath('my